In [ ]:
import cv2
from os import getenv
from pathlib import Path
import os
import re
import shutil
import zipfile

In [ ]:
input_folder = Path(getenv(
    'CROSSCOMPUTE_INPUT_FOLDER', 'batches/standard/input'))
output_folder = Path(getenv(
    'CROSSCOMPUTE_OUTPUT_FOLDER', 'batches/standard/output'))
output_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
# Extract images from zip file
def extract_images_from_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

In [ ]:
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

In [ ]:
def resize_image(image, target_width, target_height):
    """
    Resize an image to fit within the specified dimensions while maintaining the original aspect ratio.
    """
    original_height, original_width = image.shape[:2]
    aspect_ratio = original_width / original_height
    
    if original_width > original_height:
        new_width = target_width
        new_height = int(target_width / aspect_ratio)
    else:
        new_height = target_height
        new_width = int(target_height * aspect_ratio)

    resized_image = cv2.resize(image, (new_width, new_height))
    
    # If the new dimensions are smaller in any direction, pad the resized image with black color
    top_padding = (target_height - new_height) // 2
    bottom_padding = target_height - new_height - top_padding
    left_padding = (target_width - new_width) // 2
    right_padding = target_width - new_width - left_padding

    return cv2.copyMakeBorder(resized_image, top_padding, bottom_padding, left_padding, right_padding, cv2.BORDER_CONSTANT, value=[0, 0, 0])

In [ ]:
def images_to_video(image_paths, output_video_file, fps=30, duration=5, target_width=1920, target_height=1080):
    """
    Convert a set of images to a video.
    
    Parameters:
    - image_paths: list of paths to the image files
    - output_video_file: path to the output video file (e.g., 'output.mp4')
    - fps, duration, target_width, target_height as before
    """
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_file, fourcc, fps, (target_width, target_height))
    
    for img_path in image_paths:
        img = cv2.imread(img_path)
        resized_img = resize_image(img, target_width, target_height)
        
        for _ in range(fps * duration):
            out.write(resized_img)
    
    out.release()

In [ ]:
# no sorting
# output_video_file = Path(output_folder / 'output_video.mp4')

In [ ]:
# Hard coded: 
# zip_file_path = input_folder / 'Archive.zip'

# Find the first zip file in the input folder
zip_files = list(input_folder.glob('*.zip'))
if not zip_files:
    raise ValueError("No zip file found in the input folder")
zip_file_path = zip_files[0]

In [ ]:
extract_to_folder = output_folder / 'extracted_images'
extract_to_folder.mkdir(exist_ok=True)

In [ ]:
# Extract images
extract_images_from_zip(zip_file_path, extract_to_folder)

In [ ]:
# Natural Sort Function for image paths FOR INPUT Images
# image_paths = sorted([str(file_path) for file_path in input_folder.glob('*.png')], key=natural_sort_key)

In [ ]:
# For zip file
image_paths = sorted([str(file_path) for file_path in extract_to_folder.glob('*.png')], key=natural_sort_key)

In [ ]:
output_video_file = Path(output_folder / 'output_video.mp4')

In [ ]:
# Call the images_to_video function
images_to_video(image_paths, str(output_video_file), fps=30, duration=5)

In [ ]:
# Verify that the video was created and is not empty
if os.path.exists(output_video_file) and os.path.getsize(output_video_file) > 0:
    print(f"Video created successfully: {output_video_file}")
    
    # Cleanup: delete the folder with extracted images
    try:
        shutil.rmtree(extract_to_folder)
        print(f"Deleted extracted images folder: {extract_to_folder}")
    except OSError as e:
        print(f"Error deleting folder {extract_to_folder}: {e.strerror}")
else:
    print(f"Video creation failed or video file is empty: {output_video_file}")